In [3]:
import numpy as np
import pandas as pd
'''
import nltk
!pip install nltk
nltk.download('stopwords')
'''

"\nimport nltk\n!pip install nltk\nnltk.download('stopwords')\n"

In [4]:
df = pd.read_csv('IMDB Dataset.csv')
df = df[:10000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


Performing pre-prosessing steps

In [5]:
#removing tags from text -> 'review'
import re
def tag_removal(text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',text)
    return cleaned_text

df['review'] = df['review'].apply(tag_removal)

In [6]:
#lowering the text
df['review'] = df['review'].apply(lambda x:x.lower())
df['review']

0       one of the other reviewers has mentioned that ...
1       a wonderful little production. the filming tec...
2       i thought this was a wonderful way to spend ti...
3       basically there's a family where a little boy ...
4       petter mattei's "love in the time of money" is...
                              ...                        
9995    fun, entertaining movie about wwii german spy ...
9996    give me a break. how can anyone say that this ...
9997    this movie is a bad movie. but after watching ...
9998    this is a movie that was probably made to ente...
9999    smashing film about film-making. shows the int...
Name: review, Length: 10000, dtype: object

In [7]:
from nltk.corpus import stopwords
sw_list  = stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[item for item in x.split() if item not in
                                            sw_list]).apply(lambda x:' '.join(x))
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 10000, dtype: object

In [8]:

#applying stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_word(text):
    return ' '.join([ps.stem(word) for word in text.split()])

df['review'] = df['review'].apply(stem_word)
df['review']

0       one review mention watch 1 oz episod hooked. r...
1       wonder littl production. film techniqu unassum...
2       thought wonder way spend time hot summer weeke...
3       basic there' famili littl boy (jake) think the...
4       petter mattei' "love time money" visual stun f...
                              ...                        
9995    fun, entertain movi wwii german spi (juli andr...
9996    give break. anyon say "good hockey movie"? kno...
9997    movi bad movie. watch endless seri bad horror ...
9998    movi probabl made entertain middl school, earl...
9999    smash film film-making. show intens strang rel...
Name: review, Length: 10000, dtype: object

In [9]:
X = df.loc[:,'review']
y = df['sentiment']


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
#printing the dtype of each for safety purpose
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()

In [12]:
# prompt: how to fix the below error

# Assuming the error is due to X_train being a pandas Series and not a list of strings
X_train_bow = cv.fit_transform(X_train.tolist()).toarray()
X_test_bow = cv.transform(X_test.tolist()).toarray()


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8465

In [14]:
import pickle
pickle.dump(rf,open('model.pkl','wb'))

In [15]:
pickle.dump(cv,open('vectorizer.pkl','wb'))